In [95]:
import numpy as np
from random import choice,sample,randint
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from os import remove
import cProfile

In [96]:
class GA:
    def __init__(self,generation,population,expressions,scaler,thresh,cutoff):
        self.generation = generation
        self.population = population
        self.expressions = expressions
        self.history = []    
        self.scaler = scaler
        self.thresh = thresh
        self.cutoff = cutoff
    
    def fit_transform(self,X,Y):
        originalX = X.reshape(X.shape[1],X.shape[0],1)
        self.xshape = X.shape
        print(X.shape)
        tsProb = self._get_ts_prob(self.xshape[1])
        print(tsProb.shape)
        prevProb = []
        
        for i in tqdm(range(self.generation),desc = "Generation"):
            print("Fitting Generation {}".format(i+1))
            self.historyTemp = []
            memArray = np.memmap(dtype = 'float32',filename = "tempArray.array",shape = (self.population+
                                                                                         originalX.shape[0],X.shape[0],
                                                                                         1),mode = "w+")
            memArray[:originalX.shape[0],:,:] = originalX[:]
            surviveCount = originalX.shape[0]
            prob = np.append(tsProb,prevProb)
            prob = self._normalize_prob(prob)
            print("PREV prob = {} , type = {}".format(prob,prob.dtype))

            
            prevProb = []
            c = 0
            for j in tqdm(range(self.population),desc = "Population"):
                child,score,survive = self._gen_child(X,Y,prob,i)
                if(survive):
                    #print("SNR = {}".format(score))
                    child = child.reshape(len(child),1)
                    child = self.scaler.fit_transform(child)
                    memArray[surviveCount] = child
                    surviveCount+=1
                    c = c+1
                    prevProb.append(score)
            if (surviveCount==0):
                prxnt("No surviver return previous generation {}".format(i))
                return X
            X = memArray[:surviveCount,:,:]
            print("pre {}".format(X.shape))
            X = X.reshape((surviveCount,X.shape[1])).transpose()
            remove("tempArray.array")
            
            print("XSHAPE = {}".format(X.shape))
            print("SURVIVED = {}".format(c))
            self.history.append(self.historyTemp)
        return X
    
    def transform(self,X,gen = -1):
        
        originalX = X.reshape(X.shape[1],X.shape[0],1)
        
        tempHist = self.history
        if gen >0:
            tempHist = self.history[:gen:]
            
        for i,generation in tqdm(enumerate(tempHist),desc="Generation"):
            print("Transforming Generation {}".format(i+1))
            
            memArray = np.memmap(dtype = "float32"
                                 ,filename = "tempArray2.array"
                                 ,shape = (self.population+originalX.shape[0],X.shape[0],1)
                                 ,mode = "w+")
            
            memArray[:originalX.shape[0],:,:] = originalX[:]
            jdx = originalX.shape[0]
            #tempChild = np.zeros((X.shape[0],1))
            for j,population in tqdm(enumerate(generation),desc = "Population"):
                expression,idx,n = population
                operand = self._gen_operand(X,idx)
              
                child = expression.compute(operand,n)
                child = child.reshape(len(child),1)

                child = self.scaler.fit_transform(child)
                
                memArray[jdx] = child
                #tempChild = np.hstack((tempChild,child))
                jdx = jdx+1
            X = memArray[:jdx,:,:]
            #print(X.shape)
            X = X.reshape((jdx,X.shape[1])).transpose()
            #print(X.shape)
            remove("tempArray2.array")
            
        return X
    
    
    def _get_ts_prob(self,size):
        M = self.xshape[1]
        priorprob = 1.-np.exp(0.05*np.arange(M))+np.exp(0.05*M)
        priorprob = priorprob/np.sum(priorprob)
        return priorprob
    
    def get_history(self):
        return self.history
    
    def _normalize_prob(self,prob):
        prob_factor = 1 / sum(prob)
        res =  prob*prob_factor
        print("Normalize prob :{}".format(res.dtype))
        return res
        
    
    def _gen_child(self,X,Y,prob,i):
        expression = self._gen_function()
        #n_var = expression.gen_var(X)
        n_var = expression.n_var
        if (n_var == -1):
            n_var = self._gen_n(X)
        idx = self._gen_idx(X,n_var,prob)
        n = expression.gen_n()
        operand = self._gen_operand(X,idx)
        result = expression.compute(operand,n)
        survived,score = self.thresh.survived(result,Y,self.cutoff[i])
        if (survived):
            self.historyTemp.append((expression,idx,n))
            return result,score,True
        else:
            return result,score,False
    
    def _gen_n(self,X):
        n = np.random.randint(1,X.shape[1])
        print(n)
        return n

    def _gen_function(self):
        return choice(self.expressions)        
    
    def _gen_idx(self,X,n_var,prob):
        if(prob.dtype =='O'):
            print(prob)
            print(prob.dtype)
            print(X.dtype)
        return np.random.choice(a = range(0,X.shape[1]),size = n_var,p = prob)
        #return np.random.choice(a = range(0,X.shape[1]),size = n_var)
        
    def _gen_operand(self,X,idx):
        #print(X.shape)
        #print(X[:,idx].shape)
        return X[:,idx]

        
        

In [97]:
np.random.choice(range(0,10),5)

array([0, 8, 0, 9, 2])

In [98]:
## return numbers only
class Log:
    n_var = 1
    def gen_n():
        return None
    def compute(x,n):
        result = np.log(x[:,0])
        return np.nan_to_num(result)

class Add:
    n_var = 2
    def gen_n():
        return None
    def compute(x,n):
        return np.add(x[:,0],x[:,1])

class Subtract:
    n_var = 2
    def gen_n():
        return None
    def compute(x,n):
        return np.subtract(x[:,0],x[:,1])
    
class Multiply:
    n_var = 2
    def gen_n():
        return None
    def compute(x,n):
        return np.multiply(x[:,0],x[:,1])

class Divide:
    n_var = 2
    def gen_n():
        return None
    def compute(x,n):
        result = np.divide(x[:,0],x[:,1])
        return np.nan_to_num(result)

class Mean:
    n_var = -1
    def gen_n():
        return None
    def compute(x,n):
        return np.mean(x,axis = 1)

class Power:
    n_var = 1
    def gen_n():
        n = choice([1,2,3,4])
        return n
        
    def compute(x,n):
        return np.nan_to_num(x**n)

class Exp:
    n_var = 1
    def gen_n():
        return None
    
    def compute(x,n):
        return np.nan_to_num(np.exp(x))
    
class Tanh:
    n_var = 1
    def gen_n():
        return None
    
    def compute(x,n):
        return np.nan_to_num(np.tanh(x))

    
class Cos:
    n_var = 1
    
    def gen_n():
        return randint(0,90)
    
    def compute(x,n):
        return np.nan_to_num(np.cos(x+n))

class Sigmoid:
    n_var = 1
    def gen_n():
        return None
    
    def compute(x,n):
        res = 1/(1+np.exp(-x))
        return res
    



    


In [99]:
class DummpyScale():
    def fit_transform(self,x):
        return x
    
class RandomPass():
    def survived(self,x,y,t):
        sc = randint(0,1)
        return bool(sc),sc

class SNR():
        
    def survived(self, X, y,thresh):
        if len(X.shape) == 1:
            fdr = [self.__binfdr(X,y)]
        else:
            (N,M) = X.shape
            fdr = np.array([])
            for j in range(M):
                feat = X[:,j]
                
                score = self.__binfdr(feat,y)
                fdr = np.append(fdr,score)
        #print("fdr = {}:::type = {}".format(fdr,type(fdr)))
        #print("thresh = {}:::type = {}".format(thresh,type(thresh)))
        return thresh<fdr[0],fdr[0]
    
    def __binfdr(self, feat, y):
        (pluses,minuses) = np.unique(y)
        idx = np.where(y == pluses)
        x = feat[idx]
        muPlus = np.mean(x)
        varPlus = np.var(x)
        idx = np.where(y == minuses)
        x = feat[idx]
        muMinus = np.mean(x)
        varMinus = np.var(x)

        score = ((muPlus - muMinus)**2)/(varPlus + varMinus)
        return score
    

In [100]:
trainX = np.load("../Genetic_Algorithm/sa_sumdr_ts_off7.npy")
trainY = np.load("../Genetic_Algorithm/sa_sumdr_label.npy")

In [101]:
scaler = MinMaxScaler(feature_range=(0.1,1.9))
methods = [Add,Subtract,Multiply,Divide,Log,Mean,Power,Cos,Tanh,Exp,Sigmoid]
thresh = SNR()
g = GA(3,10000,methods,scaler,thresh,[0.01,0.1,0.2])

In [102]:
fitX = trainX[:10000]
fitY = trainY[:10000]

transformX = trainX[10000:20000]
transformY  =trainY[10000:20000]

In [ ]:
result = g.fit_transform(fitX,fitY)

















Generation:   0%|          | 0/3 [00:00<?, ?it/s]
















Population:   0%|          | 0/10000 [00:00<?, ?it/s]/home/paichana/miniconda3/envs/boom/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in true_divide
/home/paichana/miniconda3/envs/boom/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in true_divide
/home/paichana/miniconda3/envs/boom/lib/python3.6/site-packages/numpy/core/_methods.py:70: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims)
/home/paichana/miniconda3/envs/boom/lib/python3.6/site-packages/numpy/core/_methods.py:102: RuntimeWarning: overflow encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True)
/home/paichana/miniconda3/envs/boom/lib/python3.6/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/home/paichana/miniconda3/envs/boom/lib/p

(10000, 84)
(84,)
Fitting Generation 1
Normalize prob :float64
PREV prob = [0.01514052 0.01512888 0.01511664 0.01510378 0.01509025 0.01507603
 0.01506109 0.01504537 0.01502885 0.01501149 0.01499323 0.01497404
 0.01495386 0.01493265 0.01491035 0.01488691 0.01486227 0.01483636
 0.01480913 0.0147805  0.0147504  0.01471876 0.01468549 0.01465052
 0.01461376 0.01457511 0.01453448 0.01449177 0.01444686 0.01439966
 0.01435003 0.01429786 0.01424302 0.01418536 0.01412475 0.01406103
 0.01399404 0.01392362 0.01384959 0.01377176 0.01368994 0.01360393
 0.01351351 0.01341845 0.01331851 0.01321346 0.01310301 0.01298691
 0.01286485 0.01273653 0.01260164 0.01245982 0.01231074 0.01215401
 0.01198925 0.01181604 0.01163395 0.01144253 0.01124129 0.01102973
 0.01080732 0.01057351 0.01032772 0.01006932 0.00979767 0.0095121
 0.00921188 0.00889628 0.00856449 0.00821568 0.007849   0.00746352
 0.00705827 0.00663224 0.00618437 0.00571354 0.00521857 0.00469822
 0.00415119 0.00357611 0.00297155 0.00233599 0.00166785


















Population:   1%|          | 108/10000 [00:00<00:19, 515.01it/s]
















Population:   1%|▏         | 135/10000 [00:00<00:23, 427.76it/s]

79
55
54
48
8
44
44
79
8
26
32
42



















Population:   2%|▏         | 179/10000 [00:00<00:22, 430.61it/s]
















Population:   2%|▏         | 240/10000 [00:00<00:21, 462.19it/s]

...


















Population:  53%|█████▎    | 5336/10000 [00:31<00:27, 168.13it/s]
















Population:  54%|█████▎    | 5353/10000 [00:31<00:27, 167.83it/s]

2645
288
5365
3063



















Population:  54%|█████▎    | 5368/10000 [00:32<00:27, 167.65it/s]
















Population:  54%|█████▍    | 5401/10000 [00:32<00:27, 167.95it/s]

3359
356
165
5362



















Population:  54%|█████▍    | 5418/10000 [00:32<00:27, 167.79it/s]

In [79]:
hist = g.get_history()
len(hist),len(hist[2])

(3, 5334)

In [80]:
result2 = g.transform(fitX)

















Generation: 0it [00:00, ?it/s]
















Population: 0it [00:00, ?it/s]
















Population: 48it [00:00, 469.86it/s]

Transforming Generation 1



















Population: 94it [00:00, 445.25it/s]
















Population: 134it [00:00, 430.39it/s]
















Population: 196it [00:00, 472.05it/s]
















Population: 234it [00:00, 443.47it/s]
















Population: 289it [00:00, 458.49it/s]
















Population: 346it [00:00, 474.57it/s]
















Population: 393it [00:00, 455.01it/s]
















Population: 436it [00:00, 438.43it/s]
















Population: 497it [00:01, 454.12it/s]
















Population: 544it [00:01, 455.33it/s]
















Population: 601it [00:01, 462.93it/s]
















Population: 664it [00:01, 474.43it/s]
















Population: 717it [00:01, 472.69it/s]
















Population: 768it [00:01, 472.82it/s]
















Population: 818it [00:01, 474.24it/s]
















Population: 869it [00:01, 474.75it/s]
















Population: 931it [00:01, 482.22it/s]
















Population: 985it [00:02, 483.86it/s]
















Population: 

Population: 4320it [00:08, 511.00it/s]
















Population: 4385it [00:08, 512.30it/s]
















Population: 4452it [00:08, 514.13it/s]
















Population: 4513it [00:08, 512.16it/s]
















Population: 4568it [00:08, 511.04it/s]
















Population: 4620it [00:09, 510.19it/s]
















Population: 4670it [00:09, 508.50it/s]
















Population: 4717it [00:09, 507.68it/s]
















Population: 4774it [00:09, 507.84it/s]
















Population: 4832it [00:09, 508.50it/s]
















Population: 4887it [00:09, 508.57it/s]
















Population: 4938it [00:09, 508.56it/s]
















Population: 4989it [00:09, 508.15it/s]














Population: 4116it [01:18, 52.43it/s]
















Population: 5039it [00:09, 507.52it/s]
















Population: 5088it [00:10, 507.02it/s]
















Population: 5162it [00:10, 509.31it/s]
















Population: 5219it [00:10, 509.88it/s]
















Population: 5

Transforming Generation 2


...


















Population: 25it [00:00, 121.81it/s]
















Population: 32it [00:00, 65.92it/s] 
















Population: 40it [00:00, 64.01it/s]
















Population: 58it [00:00, 74.00it/s]
















Population: 66it [00:01, 65.56it/s]
















Population: 80it [00:01, 70.11it/s]
















Population: 90it [00:01, 68.36it/s]
















Population: 100it [00:01, 68.91it/s]
















Population: 110it [00:01, 69.66it/s]
















Population: 118it [00:01, 70.26it/s]
















Population: 130it [00:01, 71.15it/s]
















Population: 140it [00:01, 71.87it/s]
















Population: 148it [00:02, 71.62it/s]
















Population: 162it [00:02, 74.23it/s]
















Population: 171it [00:02, 73.51it/s]
















Population: 180it [00:02, 71.67it/s]
















Population: 188it [00:02, 67.54it/s]
















Population: 215it [00:02, 73.91it/s]
















Population: 254it [00:03, 83.37it/

Population: 1156it [00:13, 87.44it/s]
















Population: 1166it [00:13, 87.01it/s]
















Population: 1175it [00:13, 86.82it/s]
















Population: 1183it [00:13, 86.54it/s]
















Population: 1196it [00:13, 86.53it/s]
















Population: 1204it [00:14, 85.99it/s]
















Population: 1211it [00:14, 85.85it/s]
















Population: 1218it [00:14, 85.50it/s]
















Population: 1236it [00:14, 86.01it/s]
















Population: 1245it [00:14, 85.42it/s]
















Population: 1273it [00:14, 86.24it/s]
















Population: 1283it [00:14, 86.11it/s]
















Population: 1298it [00:15, 86.37it/s]
















Population: 1319it [00:15, 86.89it/s]
















Population: 1341it [00:15, 87.31it/s]
















Population: 1353it [00:15, 87.02it/s]
















Population: 1379it [00:15, 87.97it/s]
















Population: 1393it [00:15, 87.78it/s]
















Population: 1405it [00:16, 8

Population: 2387it [00:26, 89.75it/s]
















Population: 2407it [00:26, 90.08it/s]
















Population: 2420it [00:26, 90.05it/s]
















Population: 2432it [00:27, 89.67it/s]
















Population: 2442it [00:27, 89.39it/s]
















Population: 2451it [00:27, 88.78it/s]
















Population: 2463it [00:27, 88.89it/s]
















Population: 2485it [00:27, 89.04it/s]
















Population: 2494it [00:28, 88.94it/s]
















Population: 2505it [00:28, 89.01it/s]
















Population: 2534it [00:28, 89.72it/s]
















Population: 2550it [00:28, 89.65it/s]
















Population: 2565it [00:28, 89.82it/s]
















Population: 2579it [00:28, 89.26it/s]
















Population: 2593it [00:29, 89.22it/s]
















Population: 2610it [00:29, 89.44it/s]
















Population: 2622it [00:29, 89.47it/s]
















Population: 2633it [00:29, 89.37it/s]
















Population: 2643it [00:29, 8

Population: 3455it [00:39, 86.95it/s]
















Population: 3468it [00:39, 87.00it/s]
















Population: 3480it [00:40, 86.86it/s]
















Population: 3490it [00:40, 86.76it/s]
















Population: 3528it [00:40, 87.45it/s]
















Population: 3544it [00:40, 87.54it/s]
















Population: 3559it [00:40, 86.99it/s]
















Population: 3573it [00:41, 87.11it/s]
















Population: 3585it [00:41, 86.93it/s]
















Population: 3595it [00:41, 86.67it/s]
















Population: 3604it [00:41, 86.52it/s]
















Population: 3624it [00:41, 86.68it/s]
















Population: 3633it [00:41, 86.65it/s]
















Population: 3642it [00:42, 86.33it/s]
















Population: 3650it [00:42, 86.26it/s]
















Population: 3657it [00:42, 86.16it/s]
















Population: 3667it [00:42, 86.16it/s]
















Population: 3675it [00:42, 85.85it/s]
















Population: 3681it [00:43, 8

Population: 4736it [00:54, 87.57it/s]
















Population: 4751it [00:54, 87.55it/s]
















Population: 4764it [00:54, 87.23it/s]
















Population: 4775it [00:54, 87.20it/s]
















Population: 4788it [00:54, 87.24it/s]
















Population: 4809it [00:55, 87.38it/s]
















Population: 4829it [00:55, 87.58it/s]
















Population: 4843it [00:55, 87.58it/s]
















Population: 4856it [00:55, 87.41it/s]
















Population: 4866it [00:55, 87.29it/s]
















Population: 4897it [00:55, 87.67it/s]
















Population: 4912it [00:55, 87.73it/s]
















Population: 4934it [00:56, 87.84it/s]
















Population: 4948it [00:56, 87.86it/s]
















Population: 4961it [00:56, 87.87it/s]
















Population: 4977it [00:56, 87.93it/s]
















Population: 4989it [00:56, 87.69it/s]
















Population: 4999it [00:57, 87.59it/s]
















Population: 5008it [00:57, 8

In [82]:
result.shape,result2.shape

((10000, 5418), (10000, 5418))

In [83]:
result

memmap([[0.        , 0.        , 0.        , ..., 0.30518895, 0.4461764 ,
         0.1500092 ],
        [2.        , 1.        , 0.        , ..., 0.277003  , 0.37233484,
         0.14222531],
        [2.        , 1.        , 1.        , ..., 0.27062654, 0.32333028,
         0.1379611 ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.2632972 , 0.2512089 ,
         0.13679019],
        [0.        , 1.        , 0.        , ..., 0.33620352, 0.4029497 ,
         0.16468728],
        [0.        , 0.        , 3.        , ..., 0.2880747 , 0.21168411,
         0.14736906]], dtype=float32)

In [15]:
result2

memmap([[0.        , 0.        , 0.        , ..., 0.38545942, 0.13976792,
         0.3288119 ],
        [2.        , 1.        , 0.        , ..., 0.7042942 , 0.10442337,
         0.42501026],
        [2.        , 1.        , 1.        , ..., 0.5644307 , 0.13796279,
         0.2210809 ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.2921648 , 0.21026286,
         0.29613674],
        [0.        , 1.        , 0.        , ..., 0.38545942, 0.27560526,
         0.36016986],
        [0.        , 0.        , 3.        , ..., 0.38545942, 0.19033995,
         0.3355119 ]], dtype=float32)

In [16]:
hist = g.get_history()
len(hist),len(hist[2])

(3, 3425)

In [1]:
cProfile.run("g.fit_transform(fitX,fitY)",'myFunction.profile')

import pstats
stats = pstats.Stats("myFunction.profile")
stats.strip_dirs().sort_stats('time').print_stats()

NameError: name 'cProfile' is not defined

In [36]:
M = 1000
priorprob = 1.-np.exp(0.05*np.arange(M))+np.exp(0.05*M)
priorprob = priorprob/np.sum(priorprob)

In [20]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [223]:
a = np.array([1,2,3,4,5,6])


In [225]:
np.exp(a)

array([  2.71828183,   7.3890561 ,  20.08553692,  54.59815003,
       148.4131591 , 403.42879349])

In [29]:
np.array([Add]).dtype

dtype('O')

In [ ]:
## return numbers only
class Log:
    def gen_n(x):
        return None
    def gen_var(x):
        return 1
    def compute(x,n):
        result = np.log(x[:,0])
        return np.nan_to_num(result)

class Add:
    def gen_n(x):
        return None
    def compute(x,n):
        return np.add(x[:,0],x[:,1])
    def gen_var(x):
        return 2

class Subtract:
    def gen_n(x):
        return None
    def compute(x,n):
        return np.subtract(x[:,0],x[:,1])
    def gen_var(x):
        return 2
    
class Multiply:
    def gen_n(x):
        return None
    def compute(x,n):
        return np.multiply(x[:,0],x[:,1])
    def gen_var(x):
        return 2

class Divide:
    def gen_n(x):
        return None
    def compute(x,n):
        result = np.divide(x[:,0],x[:,1])
        return np.nan_to_num(result)
    def gen_var(x):
        return 2

class Mean:
    def gen_n(x):
        return None
    def compute(x,n):
        return np.mean(x,axis = 1)
    def gen_var(x):
        return np.random.randint(1,1000)

class Power:
    def gen_var(x):
        return 1
    def gen_n(x):
        n = choice([1,2,3,4])
        return n
    def compute(x,n):
        return np.nan_to_num(x**n)

class Exp:
    def gen_var(x):
        return 1
    def gen_n(x):
        return None
    def compute(x,n):
        return np.nan_to_num(np.exp(x))
    
class Tanh:
    def gen_var(x):
        return 1
    def gen_n(x):
        return None
    def compute(x,n):
        return np.nan_to_num(np.tanh(x))

    
class Cos:
    def gen_var(x):
        return 1
    def gen_n(x):
        return randint(0,90)
    
    def compute(x,n):
        return np.nan_to_num(np.cos(x+n))

class Sigmoid:
    def gen_var(x):
        return 1
    def gen_n(x):
        return None
    def compute(x,n):
        res = 1/(1+np.exp(-x))
        return res
    



    
